In [1]:
import ccxt
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime as dt ,timedelta
import pytz
import talib as ta
import seaborn as sns


### Exchange Setting ################################################################################
ex = ccxt.ftx({'enableRateLimit':True})
ex_b = ccxt.binance({'enableRateLimit':True})
##############################################################################


def get_data(symbols,timeframe='1d'):

    data= pd.DataFrame(ex.fetch_ohlcv(symbols,timeframe),columns=['date','open','high','low','close','volume'])
#         data.drop(columns='volume',inplace=True)
    data['date']=data['date'].apply(ex.iso8601)
    data['date']=pd.to_datetime(data['date'])
#   df['date']=df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))
#   df['date']=pd.to_datetime(df['date'])
    data.set_index('date',inplace=True)

    return data




def get_data(symbols,timeframe='1d',since=None,limit=1000,ex_mode='ftx'):
        if ex_mode =='ftx':
            ex = ccxt.ftx({'enableRateLimit':True})
        elif ex_mode =='binance':
            ex = ccxt.binance({'enableRateLimit':True})
        data= pd.DataFrame(ex.fetch_ohlcv(symbols,timeframe,since=None,limit=limit)
                         ,columns=['date','open','high','low','close','volume'])
        data['volume'] =(data['volume'].astype('float'))
        data['date']=data['date'].apply(ex.iso8601)
        data['date']=pd.to_datetime(data['date'])
        data.set_index('date',inplace=True)
        return data
    
    
    
def precision_digit(symbol:str):
    
    last =ex.fetch_ticker(symbol)['last']
    # print(type(last))
    last= str(last)
    count= last.split('.')[1]
    precision =len(count)
    return precision 



In [2]:
def create_bot_zone(starts=float, gap=float, stop_zone_sell=float, stop_zone_buy=float):
    #     starts = 1800
    #     gap =  0.05 ## zone

    #     stop_zones =500
    #     stop_zone_buy= 3200

    gap_p = starts * gap
    min_range = np.arange(starts, (stop_zone_sell), -gap_p)
    max_range = np.arange(starts, (stop_zone_buy), gap_p)
    all_range = np.concatenate((min_range, max_range), axis=None)
    df = pd.DataFrame(all_range, columns=['price'])

    df.drop_duplicates(inplace=True)
    df.sort_values('price', ascending=False, inplace=True)
    df.reset_index(inplace=True)
    df.drop(columns='index', inplace=True)

    df['diff'] = (df['price'] - starts) / starts
    return df


def rebal_plans(df, fix_values=1000, start_cash=1000, max_zone=0.5, min_zone=0.5, sub_zone=0.3):
    df['value'] = (df['diff'] * start_cash) + start_cash
    df['unit'] = 0.0
    df['adj_cash'] = 0.0
    df['free_cash'] = 0.0
    df['start_cash'] = start_cash
    df['asset_value'] = fix_values
    df['equity'] = 0.0

    #     max_zone = round(max_zone - gap,1)
    #     min_zone = round(min_zone - gap,1)
    # print(max_zone,min_zone)

    for i in range(len(df)):

        diff = df['diff'][i]
        if (diff <= -sub_zone) & (diff >= -min_zone):

            #             df['asset_value'][i] = 1200
            df['asset_value'][i] = fix_values * (1 + 0.2)

            df['unit'][i] = float(df['asset_value'][i] / df['price'][i])
            df['adj_cash'][i] = df['value'][i] - df['asset_value'][i]  # if >min <max adj else  none adjust
            df['free_cash'][i] = df['adj_cash'][i] + df['start_cash'][i]
            df['equity'][i] = df['asset_value'][i] + df['free_cash'][i]

        elif (diff >= sub_zone) & (diff <= max_zone):

            #             df['asset_value'][i] = 800
            df['asset_value'][i] = fix_values * (1 - 0.2)

            df['unit'][i] = float(df['asset_value'][i] / df['price'][i])
            df['adj_cash'][i] = df['value'][i] - df['asset_value'][i]  # if >min <max adj else  none adjust
            df['free_cash'][i] = df['adj_cash'][i] + df['start_cash'][i]
            df['equity'][i] = df['asset_value'][i] + df['free_cash'][i]

        elif (diff >= -(sub_zone)) & (diff <= sub_zone):
            #         print(df['price'][i])
            #             df['asset_value'][i] = 1000
            df['asset_value'][i] = fix_values

            df['unit'][i] = float(df['asset_value'][i] / df['price'][i])
            df['adj_cash'][i] = df['value'][i] - df['asset_value'][i]  # if >min <max adj else  none adjust
            df['free_cash'][i] = df['adj_cash'][i] + df['start_cash'][i]
            df['equity'][i] = df['asset_value'][i] + df['free_cash'][i]

    #     ## Action 2  No adj if below min% ,above max%
    for i in range(len(df)):
        diff = df['diff'][i]

        if (diff < (-min_zone)):

            #             df['unit'][i]= df[df['diff']<=(-min_zone)]['unit'].values[0]
            df['unit'][i] = df[df['diff'] >= -(min_zone)]['unit'].values[-1]
            df['asset_value'][i] = df['unit'][i] * df['price'][i]
            df['adj_cash'][i] = 0  # if >min <max adj else  none adjust
            df['free_cash'][i] = df[df['diff'] >= -(min_zone)]['free_cash'].values[-1]
            df['equity'][i] = df['asset_value'][i] + df['free_cash'][i]


        elif (diff > (max_zone)):

            df['adj_cash'][i] = 0  # if >min <max adj else  none adjust
            df['unit'][i] = df[df['diff'] <= (max_zone)]['unit'].values[0]
            df['free_cash'][i] = df[df['diff'] <= (max_zone)]['free_cash'].values[0]
            df['asset_value'][i] = (df['unit'][i] * df['price'][i])
            df['equity'][i] = df['asset_value'][i] + df['free_cash'][i]
        #     bh= fix_values+start_cash

    #     df['equity_bh'] =  bh+ (bh*df['diff'])

    df.drop(columns=['start_cash'], inplace=True)
    return df

In [3]:

starts =140
gap= 0.05
stop_zone_buy =280
stop_zone_sell = 100
gap_p = starts * gap
min_range = np.arange(starts, (stop_zone_sell), -gap_p)
max_range = np.arange(starts, (stop_zone_buy), gap_p)
all_range = np.concatenate((min_range, max_range), axis=None)
df = pd.DataFrame(all_range, columns=['price'])

df.drop_duplicates(inplace=True)
df.sort_values('price', ascending=False, inplace=True)
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)

df['diff'] = (df['price'] - starts) / starts


In [5]:
start_cash = 100000
fix_values = 200000
df['value'] = (df['diff'] * start_cash) + start_cash
df['unit'] = 0.0
df['adj_cash'] = 0.0
df['free_cash'] = 0.0
df['start_cash'] = start_cash
df['asset_value'] = fix_values
df['equity'] = 0.0
df

,price,diff,value,unit,adj_cash,free_cash,start_cash,asset_value,equity
0,273.0,0.95,195000.0,0.0,0.0,0.0,100000,200000,0.0
1,266.0,0.90,190000.0,0.0,0.0,0.0,100000,200000,0.0
2,259.0,0.85,185000.0,0.0,0.0,0.0,100000,200000,0.0
3,252.0,0.80,180000.0,0.0,0.0,0.0,100000,200000,0.0
4,245.0,0.75,175000.0,0.0,0.0,0.0,100000,200000,0.0
5,238.0,0.70,170000.0,0.0,0.0,0.0,100000,200000,0.0
6,231.0,0.65,165000.0,0.0,0.0,0.0,100000,200000,0.0
7,224.0,0.60,160000.0,0.0,0.0,0.0,100000,200000,0.0
8,217.0,0.55,155000.0,0.0,0.0,0.0,100000,200000,0.0
9,210.0,0.50,150000.0,0.0,0.0,0.0,100000,200000,0.0


In [6]:
sub_zone =0.2
min_zone  = 0.5
max_zone =0.6
for i in range(len(df)):

    diff = df['diff'][i]
    if (diff <= -sub_zone) & (diff >= -min_zone):

        #             df['asset_value'][i] = 1200
        df['asset_value'][i] = fix_values * (1 + 0.2)

        df['unit'][i] = float(df['asset_value'][i] / df['price'][i])
        df['adj_cash'][i] = df['value'][i] - df['asset_value'][i]  # if >min <max adj else  none adjust
        df['free_cash'][i] = df['adj_cash'][i] + df['start_cash'][i]
        df['equity'][i] = df['asset_value'][i] + df['free_cash'][i]

    elif (diff >= sub_zone) & (diff <= max_zone):

        #             df['asset_value'][i] = 800
        df['asset_value'][i] = fix_values * (1 - 0.2)

        df['unit'][i] = float(df['asset_value'][i] / df['price'][i])
        df['adj_cash'][i] = df['value'][i] - df['asset_value'][i]  # if >min <max adj else  none adjust
        df['free_cash'][i] = df['adj_cash'][i] + df['start_cash'][i]
        df['equity'][i] = df['asset_value'][i] + df['free_cash'][i]

    elif (diff >= -(sub_zone)) & (diff <= sub_zone):
        #         print(df['price'][i])
        #             df['asset_value'][i] = 1000
        df['asset_value'][i] = fix_values

        df['unit'][i] = float(df['asset_value'][i] / df['price'][i])
        df['adj_cash'][i] = df['value'][i] - df['asset_value'][i]  # if >min <max adj else  none adjust
        df['free_cash'][i] = df['adj_cash'][i] + df['start_cash'][i]
        df['equity'][i] = df['asset_value'][i] + df['free_cash'][i]


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is

In [125]:
# %	Price	Value	Final Equity	Unit	Diff Unit	Diff	AvgPrice	Exposure	Cash
# 0.00%	142.00	200000	200000	1408.45	0.00	0	142.00	200000.0	100000.0


In [7]:
start_cash = 100000
fix_values = 200000
start_price = 142.0


In [8]:
gap = 0.05
min_zone =  0.8
diff_loss = round(start_price*gap)
min_zone_price = start_price-(start_price*min_zone)
array_minus_zone = np.arange(start_price,min_zone_price,-diff_loss)
list_price_minus = [{'price':x} for x in array_minus_zone]
df_below_zone =pd.DataFrame([{'price':start_price,'start_value':fix_values,'cash':start_cash}])
df_below_zone

,price,start_value,cash
0,142.0,200000,100000


In [9]:
df_below_zone = df_below_zone.append(list_price_minus)
df_below_zone.drop_duplicates(subset='price',inplace=True)
df_below_zone['diff'] = fix_values- df_below_zone['start_value']
# final asset_value 
df_below_zone['final_value'] = df_below_zone['start_value']

# df.drop_duplicates()

for i in range(len(df_below_zone)):
    if i != 0 :
        df_below_zone.loc[i,'start_value'] =  df_below_zone.loc[i-1,'final_value']-(df_below_zone.loc[i-1,'final_value'] *gap)
        df_below_zone.loc[i,'diff']      =  df_below_zone.loc[i-1,'final_value'] -df_below_zone.loc[i,'start_value'] 
        df_below_zone.loc[i,'cash']      =  df_below_zone.loc[i-1,'cash'] -df_below_zone.loc[i,'diff'] 
        
        df_below_zone.loc[i,'final_value']      =  df_below_zone.loc[i,'diff']  + df_below_zone.loc[i,'start_value']
df_below_zone

,price,start_value,cash,diff,final_value
0,142.0,200000.0,100000.0,0.0,200000.0
1,135.0,190000.0,90000.0,10000.0,200000.0
2,128.0,190000.0,80000.0,10000.0,200000.0
3,121.0,190000.0,70000.0,10000.0,200000.0
4,114.0,190000.0,60000.0,10000.0,200000.0
5,107.0,190000.0,50000.0,10000.0,200000.0
6,100.0,190000.0,40000.0,10000.0,200000.0
7,93.0,190000.0,30000.0,10000.0,200000.0
8,86.0,190000.0,20000.0,10000.0,200000.0
9,79.0,190000.0,10000.0,10000.0,200000.0


In [10]:
gap = 0.1
max_zone =  1.5
diff_profit = round(start_price*gap)
max_zone_price = start_price+(start_price*max_zone)
array_profit_zone = np.arange(start_price,max_zone_price,diff_profit)

list_price_profit = [{'price':x} for x in array_profit_zone]
df_profit  =pd.DataFrame([{'price':start_price,'start_value':fix_values,'cash':start_cash}])
df_profit

,price,start_value,cash
0,142.0,200000,100000


In [11]:
array_profit_zone

array([142., 156., 170., 184., 198., 212., 226., 240., 254., 268., 282.,
       296., 310., 324., 338., 352.])

In [12]:
df_profit = df_profit.append(array_profit_zone)
df_profit.drop_duplicates(subset='price',inplace=True)
df_profit['diff'] = fix_values- df_profit['start_value']
df_profit['final_value'] = df_profit['start_value']

# final asset_value 

TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

In [13]:
zone_bot = create_bot_zone(140,0.05,100,280)
df = rebal_plans(zone_bot, fix_values=200000, start_cash=100000, max_zone=0.5, min_zone=0.5, sub_zone=0.3)
df

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is

,price,diff,value,unit,adj_cash,free_cash,asset_value,equity
0,273.0,0.95,195000.0,761.904762,0.0,90000.0,208000,298000.0
1,266.0,0.90,190000.0,761.904762,0.0,90000.0,202666,292666.0
2,259.0,0.85,185000.0,761.904762,0.0,90000.0,197333,287333.0
3,252.0,0.80,180000.0,761.904762,0.0,90000.0,192000,282000.0
4,245.0,0.75,175000.0,761.904762,0.0,90000.0,186666,276666.0
5,238.0,0.70,170000.0,761.904762,0.0,90000.0,181333,271333.0
6,231.0,0.65,165000.0,761.904762,0.0,90000.0,176000,266000.0
7,224.0,0.60,160000.0,761.904762,0.0,90000.0,170666,260666.0
8,217.0,0.55,155000.0,761.904762,0.0,90000.0,165333,255333.0
9,210.0,0.50,150000.0,761.904762,-10000.0,90000.0,160000,250000.0
